In [3]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [90]:
#function to establish connection to MySQL Community Server
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            passwd = user_password
        )
        print("Successfull Connection to MySQL Database")
    except Error as err:
        print(f"Error: '{err}'")
        
    return connection

#function to create a new database in the server using SQL commands stored in query
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
#this function is used for connecting to databases in our server      
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

#function to execute SQL queries on our database
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")
        
#function to read data from database
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")
        
#takes a python list data to insert in the tables
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [9]:
pwd = "bilal.malik1050"

In [5]:
#Establishing the connection
connection = create_server_connection("localhost", "root", pwd)

Successfull Connection to MySQL Database


In [8]:
createDatabase_query = "CREATE DATABASE SportsClub"
create_database(connection, createDatabase_query)

Database created successfully


In [10]:
db = "SportsClub"

In [14]:
#creating tables in SportsClub database
create_coach_table = """
CREATE TABLE coach (
  coach_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL
  );
 """
create_player_table = """
CREATE TABLE players (
  player_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL
);
"""
create_sport_table = """
CREATE TABLE sport (
  sport_id INT PRIMARY KEY,
  sport_name VARCHAR(40) NOT NULL,
  coach INT
);
"""
connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, create_coach_table)
execute_query(connection, create_player_table)
execute_query(connection, create_sport_table)

MySQL Database connection successful
Error: '1050 (42S01): Table 'coach' already exists'
Error: '1050 (42S01): Table 'players' already exists'
Query successful


In [15]:
#creating relationships 

alter_sport = """
ALTER TABLE sport
ADD FOREIGN KEY(coach)
REFERENCES coach(coach_id)
ON DELETE SET NULL;
"""

#table to handle many to many relationship between players and sport tables
create_plays_sport_table = """
CREATE TABLE plays_sport (
  player_id INT,
  sport_id INT,
  PRIMARY KEY(player_id, sport_id),
  FOREIGN KEY(player_id) REFERENCES players(player_id) ON DELETE CASCADE,
  FOREIGN KEY(sport_id) REFERENCES sport(sport_id) ON DELETE CASCADE
);
"""

connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, alter_sport)
execute_query(connection, create_plays_sport_table)

MySQL Database connection successful
Query successful
Query successful


In [16]:
#populating the tables

pop_coach = """
INSERT INTO coach VALUES
(1,  'James', 'Smith'),
(2, 'Stefanie',  'Martin'), 
(3, 'Steve', 'Wang'),
(4, 'Friederike',  'Müller-Rossi'),
(5, 'Isobel', 'Ivanova'),
(6, 'Niamh', 'Murphy');
"""

pop_players = """
INSERT INTO players VALUES
(101, 'Marina', 'Berg'),
(102, 'Andrea', 'Duerr'),
(103, 'Philipp', 'Probst'),
(104, 'René',  'Brandt'),
(105, 'Susanne', 'Shuster'),
(106, 'Christian', 'Schreiner'),
(107, 'Harry', 'Kim'),
(108, 'Jan', 'Nowak'),
(109, 'Pablo', 'Garcia'),
(110, 'Melanie', 'Dreschler'),
(111, 'Dieter', 'Durr'),
(112, 'Max', 'Mustermann'),
(113, 'Maxine', 'Mustermann'),
(114, 'Heiko', 'Fleischer');
"""
pop_sport = """
INSERT INTO sport VALUES
(12, 'Badminton', 1),
(13, 'Football', 6),
(14, 'Basketball', 6),
(15, 'Tennis', 6),
(16, 'Cricket', 3),
(17, 'Dodge ball', 2),
(18, 'Running', 4),
(19, 'Boxing', 1),
(20, 'Gymnastics', 5);
"""
pop_plays_sport = """
INSERT INTO plays_sport VALUES
(101, 15),
(101, 17),
(102, 17),
(103, 18),
(104, 18),
(105, 18),
(106, 13),
(107, 13),
(108, 13),
(109, 14),
(109, 15),
(110, 16),
(110, 20),
(111, 16),
(114, 12),
(112, 19),
(113, 19);
"""

connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, pop_coach)
execute_query(connection, pop_players)
execute_query(connection, pop_sport)
execute_query(connection, pop_plays_sport)


MySQL Database connection successful
Query successful
Query successful
Query successful
Query successful


In [19]:
q1 = """
SELECT sport.sport_id, sport.sport_name, coach.first_name
FROM sport
JOIN coach
ON sport.coach = coach.coach_id;
"""

connection = create_db_connection("localhost", "root", pwd, db)
results = read_query(connection, q1)
for result in results:
  print(result)

MySQL Database connection successful
(12, 'Badminton', 'James')
(19, 'Boxing', 'James')
(17, 'Dodge ball', 'Stefanie')
(16, 'Cricket', 'Steve')
(18, 'Running', 'Friederike')
(20, 'Gymnastics', 'Isobel')
(13, 'Football', 'Niamh')
(14, 'Basketball', 'Niamh')
(15, 'Tennis', 'Niamh')


In [20]:
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ["sport_id", "sport_name", "coach_first_name"]
df = pd.DataFrame(from_db, columns=columns)

In [21]:
display(df)

,sport_id,sport_name,coach_first_name
0,12,Badminton,James
1,19,Boxing,James
2,17,Dodge ball,Stefanie
3,16,Cricket,Steve
4,18,Running,Friederike
5,20,Gymnastics,Isobel
6,13,Football,Niamh
7,14,Basketball,Niamh
8,15,Tennis,Niamh


In [22]:
import requests

In [25]:
response = requests.get("https://raw.githubusercontent.com/benoitvallon/100-best-books/master/books.json")
print(response.status_code)

200


In [31]:
# create a formatted string of the Python JSON object
import json
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)
    return text
jsonData = jprint(response.json())

[
    {
        "author": "Chinua Achebe",
        "country": "Nigeria",
        "imageLink": "images/things-fall-apart.jpg",
        "language": "English",
        "link": "https://en.wikipedia.org/wiki/Things_Fall_Apart\n",
        "pages": 209,
        "title": "Things Fall Apart",
        "year": 1958
    },
    {
        "author": "Hans Christian Andersen",
        "country": "Denmark",
        "imageLink": "images/fairy-tales.jpg",
        "language": "Danish",
        "link": "https://en.wikipedia.org/wiki/Fairy_Tales_Told_for_Children._First_Collection.\n",
        "pages": 784,
        "title": "Fairy tales",
        "year": 1836
    },
    {
        "author": "Dante Alighieri",
        "country": "Italy",
        "imageLink": "images/the-divine-comedy.jpg",
        "language": "Italian",
        "link": "https://en.wikipedia.org/wiki/Divine_Comedy\n",
        "pages": 928,
        "title": "The Divine Comedy",
        "year": 1315
    },
    {
        "author": "Unknown",
   

In [47]:
db = "Library"
connection = create_server_connection("localhost", "root", pwd)
createDatabase_query = "CREATE DATABASE Library"
create_database(connection, createDatabase_query)

Successfull Connection to MySQL Database
Database created successfully


In [141]:
create_author_table = """
CREATE TABLE author (
  author_id INT PRIMARY KEY,
  name VARCHAR(60) NOT NULL
  );
 """
create_book_table = """
CREATE TABLE book (
  book_id INT PRIMARY KEY,
  title VARCHAR(60) NOT NULL,
  language VARCHAR(60) NOT NULL,
  author INT,
  pages INT,
  link VARCHAR(200) NOT NULL
);
"""
connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, create_author_table)
execute_query(connection, create_book_table)

MySQL Database connection successful
Query successful
Query successful


In [142]:
alter_book = """
ALTER TABLE book
ADD FOREIGN KEY(author)
REFERENCES author(author_id)
ON DELETE SET NULL;
"""
connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, alter_book)

MySQL Database connection successful
Query successful


In [54]:
dictData = json.loads(jsonData)

In [143]:
#creating two lists of tuples, one for author details and one for books details

auth_id = 1
book_id = 100
authors_list = []
books_list = []
check = []

for obj in dictData:
    author = obj["author"]
    check.append(author)
    #check if the author is already added to the authors_list
    if check.count(author) > 1:
        pass
    else:
        authors_list.append((auth_id, author))
        auth_id = auth_id + 1
        
        
    
def getId(name):
    for item in authors_list:
        if item[1] == name:
            return item[0]
        
for obj in dictData:
    author = obj["author"]
    language = obj["language"]
    title = obj["title"]
    pages = obj["pages"]
    link = obj["link"]
    
    aId = getId(author)
    
    books_list.append((book_id, title, language, aId, pages, link))
    book_id = book_id + 1

In [144]:
#the execute_list_query function will take the list 
#of tuples created above and the sql command to populate our tables

sql_author = '''
    INSERT INTO author (author_id, name) 
    VALUES (%s, %s)
    '''
sql_book = '''
    INSERT INTO book (book_id, title, language, author, pages, link) 
    VALUES (%s, %s, %s, %s, %s, %s)
    '''
connection = create_db_connection("localhost", "root", pwd, db)
execute_list_query(connection, sql_author, authors_list)
execute_list_query(connection, sql_book, books_list)

MySQL Database connection successful
Query successful
Query successful


In [147]:
update = """
UPDATE author 
SET name = 'Unknown Author' 
WHERE name = 'Unknown';
"""

connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, update)

MySQL Database connection successful
Query successful


In [149]:
q1 = """
SELECT *
FROM author
"""
connection = create_db_connection("localhost", "root", pwd, db)
results = read_query(connection, q1)

for result in results:
    print(result)

MySQL Database connection successful
(1, 'Chinua Achebe')
(2, 'Hans Christian Andersen')
(3, 'Dante Alighieri')
(4, 'Unknown Author')
(5, 'Jane Austen')
(6, 'Honoré de Balzac')
(7, 'Samuel Beckett')
(8, 'Giovanni Boccaccio')
(9, 'Jorge Luis Borges')
(10, 'Emily Brontë')
(11, 'Albert Camus')
(12, 'Paul Celan')
(13, 'Louis-Ferdinand Céline')
(14, 'Miguel de Cervantes')
(15, 'Geoffrey Chaucer')
(16, 'Anton Chekhov')
(17, 'Joseph Conrad')
(18, 'Charles Dickens')
(19, 'Denis Diderot')
(20, 'Alfred Döblin')
(21, 'Fyodor Dostoevsky')
(22, 'George Eliot')
(23, 'Ralph Ellison')
(24, 'Euripides')
(25, 'William Faulkner')
(26, 'Gustave Flaubert')
(27, 'Federico García Lorca')
(28, 'Gabriel García Márquez')
(29, 'Johann Wolfgang von Goethe')
(30, 'Nikolai Gogol')
(31, 'Günter Grass')
(32, 'João Guimarães Rosa')
(33, 'Knut Hamsun')
(34, 'Ernest Hemingway')
(35, 'Homer')
(36, 'Henrik Ibsen')
(37, 'James Joyce')
(38, 'Franz Kafka')
(39, 'Kālidāsa')
(40, 'Yasunari Kawabata')
(41, 'Nikos Kazantzakis')


In [150]:
q2 = """
SELECT *
FROM book
"""
connection = create_db_connection("localhost", "root", pwd, db)
results = read_query(connection, q2)

for result in results:
    print(result)

MySQL Database connection successful
(100, 'Things Fall Apart', 'English', 1, 209, 'https://en.wikipedia.org/wiki/Things_Fall_Apart\n')
(101, 'Fairy tales', 'Danish', 2, 784, 'https://en.wikipedia.org/wiki/Fairy_Tales_Told_for_Children._First_Collection.\n')
(102, 'The Divine Comedy', 'Italian', 3, 928, 'https://en.wikipedia.org/wiki/Divine_Comedy\n')
(103, 'The Epic Of Gilgamesh', 'Akkadian', 4, 160, 'https://en.wikipedia.org/wiki/Epic_of_Gilgamesh\n')
(104, 'The Book Of Job', 'Hebrew', 4, 176, 'https://en.wikipedia.org/wiki/Book_of_Job\n')
(105, 'One Thousand and One Nights', 'Arabic', 4, 288, 'https://en.wikipedia.org/wiki/One_Thousand_and_One_Nights\n')
(106, "Njál's Saga", 'Old Norse', 4, 384, 'https://en.wikipedia.org/wiki/Nj%C3%A1ls_saga\n')
(107, 'Pride and Prejudice', 'English', 5, 226, 'https://en.wikipedia.org/wiki/Pride_and_Prejudice\n')
(108, 'Le Père Goriot', 'French', 6, 443, 'https://en.wikipedia.org/wiki/Le_P%C3%A8re_Goriot\n')
(109, 'Molloy, Malone Dies, The Unnamable

In [151]:
delete_book = """
DELETE FROM book 
WHERE book_id = 104;
"""
connection = create_db_connection("localhost", "root", pwd, db)
execute_query(connection, delete_book)

MySQL Database connection successful
Query successful


In [152]:
connection = create_db_connection("localhost", "root", pwd, db)
results = read_query(connection, q2)

for result in results:
    print(result)

MySQL Database connection successful
(100, 'Things Fall Apart', 'English', 1, 209, 'https://en.wikipedia.org/wiki/Things_Fall_Apart\n')
(101, 'Fairy tales', 'Danish', 2, 784, 'https://en.wikipedia.org/wiki/Fairy_Tales_Told_for_Children._First_Collection.\n')
(102, 'The Divine Comedy', 'Italian', 3, 928, 'https://en.wikipedia.org/wiki/Divine_Comedy\n')
(103, 'The Epic Of Gilgamesh', 'Akkadian', 4, 160, 'https://en.wikipedia.org/wiki/Epic_of_Gilgamesh\n')
(105, 'One Thousand and One Nights', 'Arabic', 4, 288, 'https://en.wikipedia.org/wiki/One_Thousand_and_One_Nights\n')
(106, "Njál's Saga", 'Old Norse', 4, 384, 'https://en.wikipedia.org/wiki/Nj%C3%A1ls_saga\n')
(107, 'Pride and Prejudice', 'English', 5, 226, 'https://en.wikipedia.org/wiki/Pride_and_Prejudice\n')
(108, 'Le Père Goriot', 'French', 6, 443, 'https://en.wikipedia.org/wiki/Le_P%C3%A8re_Goriot\n')
(109, 'Molloy, Malone Dies, The Unnamable, the trilogy', 'French, English', 7, 256, 'https://en.wikipedia.org/wiki/Molloy_(novel)\n